In [2]:
import glob
import nibabel as nib
import pandas as pd
import numpy as np
import os
import h5py
from sklearn import linear_model
from scipy.stats import zscore
from scipy import stats
import matplotlib.pyplot as plt
import math
import pickle
import joblib

%autosave 5

Autosaving every 5 seconds


In [3]:
subs = ['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10',
       'sub-12', 'sub-13','sub-14', 'sub-16', 'sub-17', 'sub-18', 'sub-19', 'sub-20', 'sub-21', 'sub-22',
        'sub-23', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 
       'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38']


fmripreppath = '/data/topdown/fmriprep/fmriprep/'

In [10]:
# only use voxels that are shared across 100% of participants
count = 0

for sub in subs:
    runs = ['run-1', 'run-2', 'run-3', 'run-4']
    
    for run in runs:
        # import mask
        
        mask_fname = os.path.join(fmripreppath + sub + '/func/' + \
                  sub + '_task-listen_' + run + '_space-MNI152NLin2009cAsym_desc-aparcaseg_dseg.nii.gz')
        if os.path.exists(mask_fname):
            mask = nib.load(mask_fname)
            mask_a = mask.get_fdata()

            # mask hippocampus
            hippo_mask = (mask_a == 17) + (mask_a == 53)
            if count == 0:
                product_mask = hippo_mask
                count = 1
            else:
                product_mask = product_mask*hippo_mask

In [36]:
filename = 'pickle/hippo_mask_uniform.sav'
joblib.dump(product_mask, filename)

['pickle/hippo_mask_uniform.sav']

In [6]:
# only use voxels that are shared across 80% of participants
count = 0
sum_mask = np.zeros((78, 93, 78))

for sub in subs:
    runs = ['run-1', 'run-2', 'run-3', 'run-4']
    
    for run in runs:
        # import mask
        
        mask_fname = os.path.join(fmripreppath + sub + '/func/' + \
                  sub + '_task-listen_' + run + '_space-MNI152NLin2009cAsym_desc-aparcaseg_dseg.nii.gz')
        if os.path.exists(mask_fname):
            mask = nib.load(mask_fname)
            mask_a = mask.get_fdata()

            # mask hippocampus
            hippo_mask = (mask_a == 17) + (mask_a == 53)
            sum_mask = sum_mask + hippo_mask

In [8]:
percent_mask = sum_mask/len(subs)

In [12]:
low_values_flags = percent_mask < .8  # Where values are low
percent_mask[low_values_flags] = 0

In [17]:
percent_mask = percent_mask != 0

In [18]:
np.count_nonzero(percent_mask)

785

In [19]:
filename = 'pickle/hippo_mask_80.sav'
joblib.dump(percent_mask, filename)

['pickle/hippo_mask_80.sav']